In [126]:
import os
import pandas as pd
import numpy as np

dateCol=["Release Date","Customer Target XFD (CTXFD)","NB Requested XFD (NBRXFD)","Orig Fact Cfm XFD (OCXFD)","NB Curr Req XFD","Vendor Current Req XFD",
         "Curr Fact Cfm XFD (CCXFD)","SI Date","Start Production Dt","Customer Request Date","Original Customer Target XFD (OCTXFD)","Current Customer Target XFD (CCTXFD)",
         "Exp XFD","Booking Request Send Date","Production Complete Date","Intro Date (PPM)","Receipt Date"]

csvs=[x for x in os.listdir(".") if x.lower().endswith("csv")]

vsCol=["Customer Target XFD (CTXFD)","NB Requested XFD (NBRXFD)","Orig Fact Cfm XFD (OCXFD)","SI Date"]

def inspect(dfp,less=False):
    dfcol=dfp.columns
    susCol=[x for x in dfcol if "vs" in x.lower() and "/" not in x]
    if less==True:
        return dfp[dfp[susCol].abs().gt(250).any(axis=1)][susCol]
    else:
        return dfp[dfp[susCol].abs().gt(250).any(axis=1)]

# 0709 end todo:
1. create index
2. confirm style &rarr; jan-dec, all subordinary item give seq 1-12
3. check wrong dates in (SI vs NBRXFD) or logic
4. re-create vars
5. monthly

multi level reg.

In [19]:
# for i in [x for x in os.listdir() if x.endswith("csv")]:
#     globals()[i.split(".")[0]]=pd.read_csv(i,thousands=",")

# cj["styleSeq"]=cj.groupby("orderID").cumcount()+1
# cj[["orderID","styleSeq"]]


# check primary key:
# cj.nunique().to_dict()

# `CJ1`:

In [28]:
#orderID, product, orderqua:
cj=pd.read_csv("CJ1.csv",thousands=",").rename({"Order#":"orderID","GBU (PPM)":"product","Quantity":"orderQnty"}, axis=1)
cj.loc[cj[cj["SI Month"]=="2022/00"].index,"SI Month"]="2021/12"

cj[dateCol]=cj[dateCol].apply(pd.to_datetime)
cj.head(1)

,SI Month,Fty Country (COO),Factory Abbr,INTL/NA,Mkt Region,Market,product,Line Plan Business (PPM),orderID,PO Ln#,...,Made / Purchased,SMU?,Wid/Col,Feature Technology,World Trade Commodity,World Trade Country Code,Local Product Group Minor,Local Product Minor,Product Display Name,Closure Type
0,2021/02,CHINA,CJ,INTL,APAC,TAIWAN,Core Athletic,Core Athletic,7092294,7092294-1,...,P,NaN,D,"GORE-TEX , Synthetic/Mesh Upper, TRUFUSE, WALK...",NaN,NaN,152,Womens Walking,880 Gore-Tex,Lace


In [30]:
# @var: styleseq
cj["SI Month"] = pd.to_datetime(cj["SI Month"], format='%Y/%m')
cj=cj.sort_values(by=["Style","SI Month"])
cj['styleseq'] = cj.groupby('Style')["SI Month"].rank(method='dense').astype(int)
cj["SI Month"] = cj["SI Month"].dt.strftime('%Y/%m')
cj.loc[cj.Style==np.random.choice(cj.Style), ["SI Month","styleseq"]]

,SI Month,styleseq
89,2021/01,1
163,2021/02,2
200,2021/02,2


In [31]:
cj.loc[cj.Style==np.random.choice(cj.Style), ["SI Month","styleseq"]]

,SI Month,styleseq
42,2021/01,1
81,2021/03,2
288,2021/03,2
435,2021/05,3
726,2021/05,3
777,2021/06,4
1044,2021/08,5
1045,2021/08,5
1091,2021/11,6
1099,2021/11,6


In [63]:
# cj check:
# cj[cj["SI vs CTXFD"]>180][[x for  x in cj.columns if "CTXFD" in x][:3]]
cj.loc[2004,[x for  x in cj.columns if "CTXFD" in x][:3]]="4/3/2022"
cj.loc[325,"NB Requested XFD (NBRXFD)"]="2021-03-09"
cj.loc[392,"NB Requested XFD (NBRXFD)"]="2021-03-16"

In [67]:
# ?OnTime
for i in zip(["SI vs CTXFD Early/On-time/Late","SI vs NBRXFD Early/On-time/Late",  "SI vs OCXFD Early/On-time/Late"],["iOnTime","nOnTime","cOnTime"]):
    cj[i[1]]=cj[i[0]].map( {'Early':3, 'On-time':2, 'Late':1} )

In [ ]:
#ld_timei, ld_timen, ld_timec, prd_dayso, prd_timeo, introtime:

cj["ld_timei"]=(cj["Customer Target XFD (CTXFD)"]-cj["Release Date"]).dt.days #ideal
cj["ld_timen"]=(cj["NB Requested XFD (NBRXFD)"]-cj["Release Date"]).dt.days #brand
cj["ld_timec"]=(cj["Orig Fact Cfm XFD (OCXFD)"]-cj["Release Date"]).dt.days #confirm
cj["prd_dayso"]=(cj["Production Complete Date"]-cj["Start Production Dt"]).dt.days 
cj["prd_timeo"]=(cj["Start Production Dt"]-cj["Release Date"]).dt.days 
cj["introtime"]=(cj["Start Production Dt"]-cj["Intro Date (PPM)"]).dt.days 

# `FTV2`:

In [72]:
ftv=pd.read_csv("FTV2.csv")

C:\Users\JselZ\AppData\Local\Temp\ipykernel_13768\132380678.py:1: DtypeWarning: Columns (62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  ftv=pd.read_csv("FTV2.csv")


In [104]:
inspect(ftv)

,SI vs NBRXFD,SI vs OCXFD,SI vs CTXFD
788,364,-2,60
3109,205,205,205
3110,207,207,207
3252,196,196,192
3253,192,192,192
...,...,...,...
8812,0,0,190
8814,10,-6,186
11036,187,187,197
11037,187,187,197


In [125]:
inspect(ftv)[dateCol]

,Release Date,Customer Target XFD (CTXFD),NB Requested XFD (NBRXFD),Orig Fact Cfm XFD (OCXFD),NB Curr Req XFD,Vendor Current Req XFD,Curr Fact Cfm XFD (CCXFD),SI Date,Start Production Dt,Customer Request Date,Original Customer Target XFD (OCTXFD),Current Customer Target XFD (CCTXFD),Exp XFD,Booking Request Send Date,Production Complete Date,Intro Date (PPM),Receipt Date
788,2020/9/25,2020/11/24,2020/1/25,2021/1/25,2021/1/25,2021/1/25,2021/1/25,2021/1/23,2020/12/21,NaN,2020/11/24,2020/11/24,2021/3/18,2021/1/2,2021/1/18,NaN,2021/6/17
3109,2020/12/21,2021/7/1,2021/7/1,2021/7/1,2021/7/1,2022/1/13,2022/1/13,2022/1/22,2021/12/24,2021/7/1,2021/7/1,2021/7/1,2022/1/24,2021/7/6,2022/1/22,2021/9/1,2022/1/29
3110,2020/12/21,2021/7/1,2021/7/1,2021/7/1,2021/7/1,2022/1/14,2022/1/14,2022/1/24,2021/12/25,2021/7/1,2021/7/1,2021/7/1,2022/1/24,2021/7/6,2022/1/24,2021/9/1,2022/1/29
3252,2021/1/27,2021/7/9,2021/7/5,2021/7/5,2021/8/16,2022/1/17,2022/1/17,2022/1/17,2021/12/15,2021/7/9,2021/7/9,2021/7/9,2022/1/24,2021/12/3,2021/12/29,2023/4/1,2022/1/29
3253,2020/12/21,2021/7/9,2021/7/9,2021/7/9,2021/7/9,2022/1/17,2022/1/17,2022/1/17,2021/12/29,2021/7/9,2021/7/9,2021/7/9,2022/1/24,2021/12/3,2022/1/17,2023/1/1,2022/1/29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8812,2022/1/6,2021/12/6,2022/6/14,2022/6/14,2022/6/14,2022/6/14,2022/6/14,2022/6/14,2022/5/30,2021/12/6,2021/12/6,2021/12/6,2022/6/15,2022/5/7,2022/6/14,2022/12/25,2022/7/18
8814,2021/12/15,2021/10/11,2022/4/5,2022/4/21,2022/4/15,2022/4/15,2022/4/15,2022/4/15,2022/3/29,NaN,2021/10/11,2021/10/11,2022/4/15,2022/3/12,2022/4/14,2022/2/1,2022/4/21
11036,2022/1/26,2022/7/3,2022/7/13,2022/7/13,2023/1/18,2023/1/18,2023/1/18,2023/1/16,2022/10/6,2022/7/3,2022/7/3,2022/7/3,2023/2/13,2022/8/15,2022/11/5,2022/11/1,2023/2/18
11037,2022/1/26,2022/7/3,2022/7/13,2022/7/13,2023/1/18,2023/1/18,2023/1/18,2023/1/16,2022/10/6,2022/7/3,2022/7/3,2022/7/3,2023/2/13,2022/8/15,2022/11/4,2024/5/1,2023/2/18


# `PWI2`

In [106]:
pwi=pd.read_csv("PWI2.csv",thousands=",")

C:\Users\JselZ\AppData\Local\Temp\ipykernel_13768\647758339.py:1: DtypeWarning: Columns (51,63) have mixed types. Specify dtype option on import or set low_memory=False.
  pwi=pd.read_csv("PWI2.csv",thousands=",")


In [117]:
inspect(pwi,less=1)

,SI vs NBRXFD,SI vs OCXFD,SI vs CTXFD
7321,1461,0,-22
7548,1461,0,-22
7550,1461,0,-20
7551,1461,0,-22
7552,1461,0,-22
7566,1484,23,1
7946,301,301,352
10032,45,31,208
15209,204,204,204
22657,0,-273,-1


In [140]:
inspect(pwi)[vsCol]

,Customer Target XFD (CTXFD),NB Requested XFD (NBRXFD),Orig Fact Cfm XFD (OCXFD),SI Date
7321,9/30/2021,9/8/2021,9/8/2021,9/8/2021
7548,9/30/2021,9/8/2021,9/8/2021,9/8/2021
7550,9/28/2021,9/8/2021,9/8/2021,9/8/2021
7551,9/30/2021,9/8/2021,9/8/2021,9/8/2021
7552,9/30/2021,9/8/2021,9/8/2021,9/8/2021
7566,9/30/2021,9/8/2021,9/8/2021,10/1/2021
7946,9/25/2021,9/8/2021,11/15/2021,9/12/2022
22657,1/11/2023,1/10/2023,10/10/2023,1/10/2023


In [115]:
pwi.loc[[7321, 7548, 7550, 7551, 7552, 7566, 7946], "NB Requested XFD (NBRXFD)"]="9/8/2021"

# `PYV`

In [119]:
pyv=pd.read_csv("PYV.csv",thousands=",")

In [133]:
inspect(pyv)[dateCol]

,Release Date,Customer Target XFD (CTXFD),NB Requested XFD (NBRXFD),Orig Fact Cfm XFD (OCXFD),NB Curr Req XFD,Vendor Current Req XFD,Curr Fact Cfm XFD (CCXFD),SI Date,Start Production Dt,Customer Request Date,Original Customer Target XFD (OCTXFD),Current Customer Target XFD (CCTXFD),Exp XFD,Booking Request Send Date,Production Complete Date,Intro Date (PPM),Receipt Date
378,10/28/2020,1/18/2020,2/22/2021,2/22/2021,3/6/2021,3/6/2021,3/6/2021,2/22/2021,1/29/2021,1/18/2020,1/18/2020,1/18/2020,3/12/2021,2/5/2021,2/22/2021,5/1/2021,4/14/2021
3292,2/26/2021,6/21/2021,8/23/2021,8/23/2021,8/23/2021,4/19/2022,4/19/2022,4/18/2022,3/30/2022,6/21/2021,6/21/2021,6/21/2021,4/25/2022,3/20/2022,4/18/2022,6/25/2021,5/27/2022
3293,2/26/2021,6/21/2021,8/23/2021,8/23/2021,8/23/2021,4/19/2022,4/19/2022,4/12/2022,3/30/2022,6/21/2021,6/21/2021,6/21/2021,4/25/2022,3/20/2022,4/12/2022,6/25/2021,5/27/2022
3294,2/26/2021,6/21/2021,8/23/2021,8/23/2021,8/23/2021,4/19/2022,4/19/2022,4/19/2022,3/28/2022,6/21/2021,6/21/2021,6/21/2021,4/25/2022,3/20/2022,4/19/2022,6/25/2021,5/27/2022
3295,2/26/2021,6/21/2021,8/23/2021,8/23/2021,8/23/2021,4/19/2022,4/19/2022,4/16/2022,3/28/2022,6/21/2021,6/21/2021,6/21/2021,4/25/2022,3/20/2022,4/16/2022,6/25/2021,5/27/2022
3311,2/26/2021,6/7/2021,8/16/2021,8/16/2021,8/16/2021,4/12/2022,4/12/2022,4/5/2022,3/26/2022,6/7/2021,6/7/2021,6/7/2021,4/26/2022,2/28/2022,4/5/2022,6/25/2021,5/28/2022
3312,2/26/2021,6/7/2021,8/16/2021,8/16/2021,8/16/2021,4/25/2022,4/25/2022,4/21/2022,3/26/2022,6/7/2021,6/7/2021,6/7/2021,5/4/2022,2/28/2022,4/21/2022,6/25/2021,6/6/2022
3395,2/26/2021,6/14/2021,8/16/2021,8/16/2021,8/16/2021,3/17/2022,3/17/2022,3/16/2022,3/8/2022,6/14/2021,6/14/2021,6/14/2021,3/26/2022,2/28/2022,3/16/2022,6/25/2021,4/27/2022
3396,2/26/2021,6/14/2021,8/16/2021,8/16/2021,8/16/2021,3/17/2022,3/17/2022,3/16/2022,3/8/2022,6/14/2021,6/14/2021,6/14/2021,3/26/2022,2/28/2022,3/16/2022,6/25/2021,4/27/2022
3397,2/26/2021,6/14/2021,8/16/2021,8/16/2021,8/16/2021,4/19/2022,4/19/2022,4/14/2022,6/11/2021,6/14/2021,6/14/2021,6/14/2021,4/22/2022,3/17/2022,4/14/2022,6/25/2021,5/24/2022


# `SA1`

In [122]:
sa=pd.read_csv("SA1.csv",thousands=",")

C:\Users\JselZ\AppData\Local\Temp\ipykernel_13768\1505989697.py:1: DtypeWarning: Columns (62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  sa=pd.read_csv("SA1.csv",thousands=",")


In [129]:
inspect(sa, less=1)

,SI vs NBRXFD,SI vs OCXFD,SI vs CTXFD
11,360,-6,-10
13,362,-4,-8
2473,255,255,255
2479,251,251,251
2481,251,251,251
2534,251,251,251
2536,252,252,252
2591,275,275,275
2830,267,267,267
2831,267,267,267
